<a href="https://colab.research.google.com/github/LuisaPolicarpo/Project3/blob/main/Project3_Recomendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
#Import table
df_inovmovie = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/Copy_movies_imdb.pickle')
df_inovmovie

,primaryName,tconst,genres,primaryTitle,averageRating,numVotes,primaryProfession,titleType,startYear,wheighted_IMDB
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,5.7,1922,cinematographer,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Short,Carmencita,5.7,1922,cinematographer,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,5.7,1922,director,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Short,Carmencita,5.7,1922,director,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,5.7,1922,producer,short,1894,6.278611
...,...,...,...,...,...,...,...,...,...,...
3444278,Oliver Bickers,tt9916544,Drama,My Sweet Prince,6.9,60,actor,short,2019,6.324476
3444278,Oliver Bickers,tt9916544,Short,My Sweet Prince,6.9,60,actor,short,2019,6.324476
3444279,Ganesh Vasant Patil,tt9916730,\N,6 Gunn,9.2,7,actor,movie,2017,6.323900
3444281,Ujjwala Gawde,tt9916730,\N,6 Gunn,9.2,7,producer,movie,2017,6.323900


In [ ]:
df_inovmovie['startYear'].unique()

array(['1894', '1893', '1896', '1899', '1903', '1897', '1895', '1898',
       '1892', '1891', '2018', '1890', '1901', '1878', '1877', '1979',
       '1965', '1910', '1914', '1942', '1940', '1946', '1913', '1922',
       '1911', '2019', '2015', '1904', '1920', '1933', '1929', '1905',
       '1900', '1902', '1945', '1996', '1966', '1874', '1909', '1907',
       '1908', '1906', '1912', '1939', '1978', '1921', '1915', '1919',
       '1918', '1923', '1924', '1925', '1928', '1951', '1934', '1917',
       '1926', '1927', '1930', '1931', '1932', '1935', '1936', '1937',
       '1938', '1941', '1944', '1947', '1949', '1955', '1956', '1952',
       '1991', '1916', '2003', '2004', '1992', '1948', '1983', '1950',
       '2020', '1953', '1959', '1960', '1964', '1967', '1972', '1974',
       '1976', '1977', '1984', '2002', '2005', '1982', '2006', '1980',
       '2021', '2012', '2013', '2014', '2007', '1969', '1990', '2009',
       '2022', '2011', '2016', '2017', '1943', '1954', '1957', '1958',
      

In [ ]:
condition = df_inovmovie.loc[df_inovmovie['startYear'] != '\\N']


In [ ]:
#drop averageRating, numVotes
condition.drop(['averageRating', 'numVotes'], axis = 1, inplace = True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#Genre get dummies

condition = pd.concat([condition , condition['genres'].str.get_dummies()], 
          axis = 1)

In [ ]:
condition

,primaryName,tconst,genres,primaryTitle,primaryProfession,titleType,startYear,wheighted_IMDB,Action,Adult,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,cinematographer,short,1894,6.278611,0,0,...,0,0,0,0,0,0,0,0,0,0
1,William K.L. Dickson,tt0000001,Short,Carmencita,cinematographer,short,1894,6.278611,0,0,...,0,0,0,1,0,0,0,0,0,0
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,director,short,1894,6.278611,0,0,...,0,0,0,0,0,0,0,0,0,0
1,William K.L. Dickson,tt0000001,Short,Carmencita,director,short,1894,6.278611,0,0,...,0,0,0,1,0,0,0,0,0,0
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,producer,short,1894,6.278611,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444278,Oliver Bickers,tt9916544,Drama,My Sweet Prince,actor,short,2019,6.324476,0,0,...,0,0,0,0,0,0,0,0,0,0
3444278,Oliver Bickers,tt9916544,Short,My Sweet Prince,actor,short,2019,6.324476,0,0,...,0,0,0,1,0,0,0,0,0,0
3444279,Ganesh Vasant Patil,tt9916730,\N,6 Gunn,actor,movie,2017,6.323900,0,0,...,0,0,0,0,0,0,0,0,0,1
3444281,Ujjwala Gawde,tt9916730,\N,6 Gunn,producer,movie,2017,6.323900,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# condition_g = pd.pivot_table(data = condition, values = condition[['primaryTitle', 'startYear', 'wheighted_IMDB', 'Action', 'Adult',
#        'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
#        'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
#        'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
#        'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War',
#        'Western']], index = ['primaryTitle'], aggfunc= {'startYear' : 'mean', 'wheighted_IMDB' : 'mean', 'Action' : 'max', 'Adult': 'max',
#        'Adventure' : 'max', 'Animation' : 'max', 'Biography' : 'max', 'Comedy' : 'max', 'Crime' : 'max', 'Documentary' : 'max',
#        'Drama' : 'max', 'Family' : 'max', 'Fantasy' : 'max', 'Film-Noir' : 'max', 'Game-Show' : 'max', 'History' : 'max',
#        'Horror' : 'max', 'Music' : 'max', 'Musical' : 'max', 'Mystery' : 'max', 'News' : 'max', 'Reality-TV' : 'max',
#        'Romance' : 'max', 'Sci-Fi' : 'max', 'Short' : 'max', 'Sport' : 'max', 'Talk-Show' : 'max', 'Thriller' : 'max', 'War' : 'max',
#        'Western' : 'max'})
# condition_g

In [ ]:
condition_g = condition.groupby('primaryTitle').max().sort_values(by = 'wheighted_IMDB', ascending = False)

In [ ]:
condition_gi = condition_g.reset_index().reset_index()
condition_gi.head(10)

,index,primaryTitle,primaryName,tconst,genres,primaryProfession,titleType,startYear,wheighted_IMDB,Action,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N
0,0,The Shawshank Redemption,William Sadler,tt0111161,Drama,producer,movie,1994,9.272273,0,...,0,0,0,0,0,0,0,0,0,0
1,1,The Godfather,William Reynolds,tt0460136,Short,producer,short,1972,9.161494,0,...,0,0,0,1,0,0,0,0,0,0
2,2,The Dark Knight,T.C. Christensen,tt1213819,Short,producer,short,2008,8.974812,1,...,0,0,0,1,0,0,0,0,0,0
3,3,The Lord of the Rings: The Return of the King,Viggo Mortensen,tt0167260,Drama,producer,movie,2003,8.963991,1,...,0,0,0,0,0,0,0,0,0,0
4,4,Schindler's List,Thomas Keneally,tt0108052,History,producer,movie,1993,8.951217,0,...,0,0,0,0,0,0,0,0,0,0
5,5,The Godfather Part II,Robert Duvall,tt0071562,Drama,producer,movie,1974,8.948021,0,...,0,0,0,0,0,0,0,0,0,0
6,6,12 Angry Men,Sidney Lumet,tt0050083,Drama,producer,movie,1957,8.917393,0,...,0,0,0,0,0,0,0,0,0,0
7,7,Pulp Fiction,Uma Thurman,tt0110912,Drama,producer,movie,1994,8.868730,0,...,0,0,0,0,0,0,0,0,0,0
8,8,Inception,Wally Pfister,tt7321322,Short,producer,short,2014,8.773730,1,...,0,0,1,1,0,0,0,0,0,0
9,9,Fight Club,Zach Grenier,tt0137523,Drama,producer,movie,1999,8.770918,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Clean the nans

# condition = df_inovmovie['Crime'] == r'\\N'
# condition.apply(lambda x : '1' if x == 1 else 0).sum()

In [ ]:
# df_inovmovie['startYear'].str.contains(r'\\N', regex=True).sum()

In [ ]:
# df_inovmovie['startYear_n'] = df_inovmovie['startYear'].apply(lambda x : np.nan if x == r'\\N' else x)


In [ ]:
# df_inovmovie.dropna(subset=['startYear_n'], inplace = True)
# # df_inovmovie['startYear_n'].isna().sum()

In [ ]:
# df_inovmovie['startYear_n'].isna().sum()

In [ ]:
#convert date type
# import datetime

# df_inovmovie['startYear'] = pd.to_datetime(df_inovmovie['startYear'], format='%Y')

condition_g['startYear'].astype('int')


primaryTitle
The Shawshank Redemption                         1994
The Godfather                                    1972
The Dark Knight                                  2008
The Lord of the Rings: The Return of the King    2003
Schindler's List                                 1993
                                                 ... 
Disaster Movie                                   2008
Justin Bieber: Never Say Never                   2011
Radhe                                            2021
Reis                                             2017
Sadak 2                                          2020
Name: startYear, Length: 358326, dtype: int64

In [ ]:
#NList of columns to choose from for the model
condition_gi.columns

Index(['primaryTitle', 'primaryName', 'tconst', 'genres', 'primaryProfession',
       'titleType', 'startYear', 'wheighted_IMDB', 'Action', 'Adult',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War',
       'Western', '\N'],
      dtype='object')

In [ ]:
#KNN model

X = condition_gi[['startYear', 'wheighted_IMDB',
       'Adventure']]

model = NearestNeighbors(n_neighbors=5).fit(X)

In [ ]:
name = ['The Shawshank Redemption']

array1, array2 = model.kneighbors(condition_gi.loc[condition_gi['primaryTitle'].isin(name), ['startYear', 'wheighted_IMDB', 'Adventure']])
array1, array2

(array([[0.        , 0.40354234, 0.50196728, 0.79117119, 0.818433  ]]),
 array([[ 0,  7, 10, 29, 34]]))

In [ ]:
list_1 = array1.tolist()
list_2 = array2.tolist()

flat_list1 = list(np.concatenate(list_1).flat)
flat_list2 = list(np.concatenate(list_2).flat)

d = {'Distance': flat_list1,'index': flat_list2}
df12 = pd.DataFrame(d)
df12

dfnl_df12 = pd.merge(condition_gi, df12, how='inner', on=["index", "index"])
dfnl_df12.sort_values(by = 'Distance').head(5)

,index,primaryTitle,primaryName,tconst,genres,primaryProfession,titleType,startYear,wheighted_IMDB,Action,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N,Distance
0,0,The Shawshank Redemption,William Sadler,tt0111161,Drama,producer,movie,1994,9.272273,0,...,0,0,0,0,0,0,0,0,0,0.000000
1,7,Pulp Fiction,Uma Thurman,tt0110912,Drama,producer,movie,1994,8.868730,0,...,0,0,0,0,0,0,0,0,0,0.403542
2,10,Forrest Gump,Winston Groom,tt0109830,Romance,producer,movie,1994,8.770305,0,...,1,0,0,0,0,0,0,0,0,0.501967
3,29,It's a Wonderful Life,Tony Ka Fai Leung,tt0109524,Fantasy,producer,movie,1994,8.481102,0,...,0,0,0,0,0,0,0,0,0,0.791171
4,34,Léon: The Professional,Thierry Arbogast,tt0110413,Drama,producer,movie,1994,8.453840,1,...,0,0,0,0,0,0,0,0,0,0.818433


In [ ]:
#Remove the movie that is to be recomended from the list
name = ('The Shawshank Redemption')

condition_gi_nn = condition_gi[condition_gi['primaryTitle'].str.contains(name) == False]
# condition_gi_nn = condition_gi.loc[condition_gi['primaryTitle'] != name]

name1 = ['The Shawshank Redemption']

X1 = condition_gi_nn[['startYear', 'wheighted_IMDB',
       'Adventure']]

model = NearestNeighbors(n_neighbors=5).fit(X1)

array1, array2 = model.kneighbors(condition_gi.loc[condition_gi['primaryTitle'].isin(name1), ['startYear', 'wheighted_IMDB', 'Adventure']])
array1, array2

list_1 = array1.tolist()
list_2 = array2.tolist()

flat_list1 = list(np.concatenate(list_1).flat)
flat_list2 = list(np.concatenate(list_2).flat)

d = {'Distance': flat_list1,'index': flat_list2}
df12 = pd.DataFrame(d)
df12

dfnl_df12 = pd.merge(condition_gi_nn, df12, how='inner', on=["index", "index"])
dfnl_df12.sort_values(by = 'Distance').head(5)

,index,primaryTitle,primaryName,tconst,genres,primaryProfession,titleType,startYear,wheighted_IMDB,Action,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N,Distance
1,6,12 Angry Men,Sidney Lumet,tt0050083,Drama,producer,movie,1957,8.917393,0,...,0,0,0,0,0,0,0,0,0,0.403542
2,9,Fight Club,Zach Grenier,tt0137523,Drama,producer,movie,1999,8.770918,0,...,0,0,0,0,0,0,0,0,0,0.501967
3,28,Life Is Beautiful,Zita Kabátová,tt3881136,Romance,producer,short,2022,8.520547,0,...,1,0,0,0,0,0,0,0,0,0.791171
4,33,Back to the Future,Robert Zemeckis,tt0088763,Sci-Fi,producer,movie,1985,8.455415,0,...,0,1,0,0,0,0,0,0,0,0.818433
0,3,The Lord of the Rings: The Return of the King,Viggo Mortensen,tt0167260,Drama,producer,movie,2003,8.963991,1,...,0,0,0,0,0,0,0,0,0,1.050274
